In [1]:
import numpy as np
import pandas as pd

In [2]:
products = pd.read_csv("products.csv")
orders = pd.read_csv("orders.csv")
prior = pd.read_csv('order_products__prior.csv')

In [3]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [5]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [11]:
prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [18]:
food = pd.read_csv('food_fact.csv',low_memory=False)

In [19]:
food.head()

,Unnamed: 0,code,product_name,brands,country,ingredients_text,additives,nutrition_grade_fr,energy,fat,saturated-fat,carbohydrates,sugars,fiber,proteins,salt,sodium,nutrition-score-fr,serving_size_numeric,serving_size_units
0,67,0000000032117,Tricolor Tortellini,Napoleon,United States,"Egg pasta (refined durum semolina wheat flour,...",0.0,c,1611.0,9.23,3.08,60.00,1.54,3.1,13.85,1.13284,0.446,7.0,65.0,g
1,72,0000000033060,10-Grain Pancake Mix,Bob's Red Mill,United States,"Wheat, rye, triticale, oat, corn, barley, soy ...",2.0,c,1297.0,1.72,0.86,60.34,5.17,6.9,13.79,2.75844,1.086,9.0,58.0,g
2,175,0000000069601,Fresh Organic Carrots,Bunny-Luv,United States,Fresh organic carrots,0.0,a,159.0,0.00,0.00,8.97,6.41,2.6,1.28,0.19558,0.077,-2.0,78.0,g
3,176,0000000290616,Salade Cesar,Kirkland Signature,Canada,INGREDIENTS : LAITUE. VINAIGRETTE (HUILE DE CA...,0.0,c,1210.0,12.00,7.00,23.00,0.00,2.0,22.00,2.16000,0.850,6.0,100.0,g
4,182,0000001938067,Chaussons tressés aux pommes,Kirkland Signature,Canada,"Ingrédients : Pâte (farine, margarines d'huile...",5.0,c,1090.0,10.70,2.00,38.70,24.70,2.0,3.33,0.64700,0.255,9.0,150.0,g


In [38]:
filtered_products = products[products['product_name'].isin(food['product_name'])]

In [39]:
filtered_products.shape

(4520, 4)

In [41]:
data = pd.merge(
    prior[['order_id', 'product_id']], 
    orders[['order_id', 'user_id']], 
    on='order_id',                  
    how='inner'
)

In [43]:
data = pd.merge(
    data, 
    products[['product_id', 'product_name']], 
    on='product_id', 
    how='left'
)

In [45]:
data = pd.merge(
    data, 
    food[['code', 'product_name']], 
    on='product_name', 
    how='left'
)

In [47]:
data = data.dropna()

In [48]:
data = data.drop(['product_id', 'product_name'], axis=1)

(95691031, 5)

In [50]:
aggregated_data = data.groupby(['user_id', 'order_id']).agg({
    'code': lambda x: ', '.join(x)
}).reset_index()

In [55]:
aggregated_data['new_order_id'] = aggregated_data.groupby('user_id').cumcount() + 1

In [62]:
aggregated_data

,user_id,order_id,code,shopping_list_id,new_order_id
0,1,431534,"0095301133830, 0095301133861, 0620221290051, 0...",1,1
1,1,473747,"0095301133830, 0095301133861, 0620221290051, 0...",2,2
2,1,550135,"0095301133830, 0095301133861, 0620221290051, 0...",3,3
3,1,2254736,"0095301133830, 0095301133861, 0620221290051, 0...",4,4
4,1,2295261,"0041190463688, 0717544207825, 0717544207832, 0...",5,5
...,...,...,...,...,...
2669813,206209,1854736,"0011110428561, 0013086004657, 0014000039854, 0...",5,5
2669814,206209,2129269,"0011110428561, 0013086004657, 0014000039854, 0...",6,6
2669815,206209,2977660,"0011110816337, 0011110816368, 0011110845320, 0...",7,7
2669816,206209,3154581,"0020113001855, 0023933930831, 0040822341349, 0...",8,8


In [63]:
aggregated_data['shopping_list_id'] = aggregated_data['user_id'].astype(str) + '_' + aggregated_data['new_order_id'].astype(str)

In [65]:
aggregated_data = aggregated_data.drop(['order_id', 'new_order_id'], axis=1)
columns_order = ['user_id', 'shopping_list_id', 'code']
aggregated_data = aggregated_data[columns_order]

In [68]:
aggregated_data

,user_id,shopping_list_id,code
0,1,1_1,"0095301133830, 0095301133861, 0620221290051, 0..."
1,1,1_2,"0095301133830, 0095301133861, 0620221290051, 0..."
2,1,1_3,"0095301133830, 0095301133861, 0620221290051, 0..."
3,1,1_4,"0095301133830, 0095301133861, 0620221290051, 0..."
4,1,1_5,"0041190463688, 0717544207825, 0717544207832, 0..."
...,...,...,...
2669813,206209,206209_5,"0011110428561, 0013086004657, 0014000039854, 0..."
2669814,206209,206209_6,"0011110428561, 0013086004657, 0014000039854, 0..."
2669815,206209,206209_7,"0011110816337, 0011110816368, 0011110845320, 0..."
2669816,206209,206209_8,"0020113001855, 0023933930831, 0040822341349, 0..."
